# Magentic Multi-Agent Orchestration

## Overview

This notebook demonstrates **Magentic orchestration** - an advanced multi-agent coordination pattern that automatically manages task decomposition, agent selection, and result synthesis. Unlike concurrent or sequential patterns, Magentic uses an intelligent orchestrator that:

1. **Plans** how to decompose the task
2. **Delegates** subtasks to appropriate agents
3. **Monitors** progress and adapts the plan
4. **Synthesizes** final results

### Key Concepts:

1. **MagenticBuilder**: High-level API for multi-agent orchestration
2. **Standard Manager**: Built-in orchestrator with planning capabilities
3. **Specialized Agents**: Domain experts (Researcher, Coder)
4. **Streaming Callbacks**: Real-time event monitoring
5. **Event Types**: Orchestrator messages, agent deltas, agent messages, final result

### Architecture:

```
User Task
    ↓
Magentic Orchestrator (Standard Manager)
    ↓
Plan Generation
    ↓
┌─────────────────────────┐
│  Delegate to Agents     │
│  ├── ResearcherAgent    │
│  └── CoderAgent         │
└─────────────────────────┘
    ↓
Monitor & Adapt
    ↓
Synthesize Final Result
```

### Workflow Parameters:

- **max_round_count**: Maximum orchestration rounds (default: 10)
- **max_stall_count**: Retries when progress stalls (default: 3)
- **max_reset_count**: Full plan resets allowed (default: 2)

## Prerequisites

- OpenAI API key configured: `OPENAI_API_KEY` environment variable
- Agent Framework installed: `pip install agent-framework`
- Special models for some agents:
  - ResearcherAgent: `gpt-4o-search-preview` (web search capability)
  - CoderAgent: OpenAI Assistants with code interpreter

## Setup and Imports

In [ ]:
import asyncio
import logging

import os
from dotenv import load_dotenv
from agent_framework import (
    ChatAgent,
    HostedCodeInterpreterTool,
    MagenticAgentDeltaEvent,
    MagenticAgentMessageEvent,
    MagenticBuilder,
    MagenticCallbackEvent,
    MagenticCallbackMode,
    MagenticFinalResultEvent,
    MagenticOrchestratorMessageEvent,
    WorkflowOutputEvent,
)
from agent_framework.openai import OpenAIChatClient, OpenAIResponsesClient

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)
# Load environment variables from .env file
load_dotenv('../../.env')


## Create Specialized Agents

### ResearcherAgent
- Uses `gpt-4o-search-preview` model for web search
- Specializes in finding information without computation
- Can search current data sources

### CoderAgent
- Uses OpenAI Assistants with Code Interpreter
- Executes Python code for analysis and computation
- Provides detailed analysis processes

In [ ]:
async def run_magentic_workflow() -> None:
    researcher_agent = ChatAgent(
        name="ResearcherAgent",
        description="Specialist in research and information gathering",
        instructions=(
            "You are a Researcher. You find information without additional computation or quantitative analysis."
        ),
        # This agent requires the gpt-4o-search-preview model to perform web searches.
        chat_client=OpenAIChatClient(model_id="gpt-4o-search-preview"),
    )

    coder_agent = ChatAgent(
        name="CoderAgent",
        description="A helpful assistant that writes and executes code to process and analyze data.",
        instructions="You solve questions using code. Please provide detailed analysis and computation process.",
        chat_client=OpenAIResponsesClient(),
        tools=HostedCodeInterpreterTool(),
    )

## Define Event Callback Handler

The unified callback processes all workflow events:

### Event Types:

1. **MagenticOrchestratorMessageEvent**: Planning and coordination messages
2. **MagenticAgentDeltaEvent**: Streaming token updates from agents
3. **MagenticAgentMessageEvent**: Complete agent responses
4. **MagenticFinalResultEvent**: Final synthesized result

### Callback Benefits:
- Real-time visibility into orchestration
- Incremental streaming updates
- Clear separation of orchestrator vs. agent activity

In [ ]:
    # State variables for streaming display
    last_stream_agent_id: str | None = None
    stream_line_open: bool = False

    # Unified callback for all events
    async def on_event(event: MagenticCallbackEvent) -> None:
        """
        Process events emitted by the workflow.
        Events include: orchestrator messages, agent delta updates, agent messages, and final result events.
        """
        nonlocal last_stream_agent_id, stream_line_open
        
        if isinstance(event, MagenticOrchestratorMessageEvent):
            print(f"\n[ORCH:{event.kind}]\n\n{getattr(event.message, 'text', '')}\n{'-' * 26}")
            
        elif isinstance(event, MagenticAgentDeltaEvent):
            if last_stream_agent_id != event.agent_id or not stream_line_open:
                if stream_line_open:
                    print()
                print(f"\n[STREAM:{event.agent_id}]: ", end="", flush=True)
                last_stream_agent_id = event.agent_id
                stream_line_open = True
            print(event.text, end="", flush=True)
            
        elif isinstance(event, MagenticAgentMessageEvent):
            if stream_line_open:
                print(" (final)")
                stream_line_open = False
                print()
            msg = event.message
            if msg is not None:
                response_text = (msg.text or "").replace("\n", " ")
                print(f"\n[AGENT:{event.agent_id}] {msg.role.value}\n\n{response_text}\n{'-' * 26}")
                
        elif isinstance(event, MagenticFinalResultEvent):
            print("\n" + "=" * 50)
            print("FINAL RESULT:")
            print("=" * 50)
            if event.message is not None:
                print(event.message.text)
            print("=" * 50)

## Build Magentic Workflow

### Configuration:

- **`.participants(...)`**: Define available agents (keyword arguments)
- **`.on_event(...)`**: Register callback with streaming mode
- **`.with_standard_manager(...)`**: Configure orchestrator
  - `chat_client`: LLM for planning and coordination
  - `max_round_count`: Maximum orchestration cycles
  - `max_stall_count`: Retries when no progress
  - `max_reset_count`: Full plan restarts allowed

In [ ]:
    print("\nBuilding Magentic Workflow...")

    workflow = (
        MagenticBuilder()
        .participants(researcher=researcher_agent, coder=coder_agent)
        .on_event(on_event, mode=MagenticCallbackMode.STREAMING)
        .with_standard_manager(
            chat_client=OpenAIChatClient(),
            max_round_count=10,
            max_stall_count=3,
            max_reset_count=2,
        )
        .build()
    )

## Define Complex Task

This task requires:
1. **Research**: Finding energy consumption data for ML models
2. **Computation**: Calculating CO2 emissions
3. **Analysis**: Comparing across different architectures
4. **Synthesis**: Recommendations by task type

The orchestrator will automatically decompose this into subtasks and delegate appropriately.

In [ ]:
    task = (
        "I am preparing a report on the energy efficiency of different machine learning model architectures. "
        "Compare the estimated training and inference energy consumption of ResNet-50, BERT-base, and GPT-2 "
        "on standard datasets (e.g., ImageNet for ResNet, GLUE for BERT, WebText for GPT-2). "
        "Then, estimate the CO2 emissions associated with each, assuming training on an Azure Standard_NC6s_v3 "
        "VM for 24 hours. Provide tables for clarity, and recommend the most energy-efficient model "
        "per task type (image classification, text classification, and text generation)."
    )

    print(f"\nTask: {task}")
    print("\nStarting workflow execution...")

## Execute Workflow with Streaming

The workflow:
1. Generates execution plan
2. Delegates to agents as needed
3. Monitors progress and adapts
4. Synthesizes final result
5. Completes when idle or max rounds reached

In [ ]:
    try:
        output: str | None = None
        async for event in workflow.run_stream(task):
            print(event)
            if isinstance(event, WorkflowOutputEvent):
                output = str(event.data)

        if output is not None:
            print(f"Workflow completed with result:\n\n{output}")

    except Exception as e:
        print(f"Workflow execution failed: {e}")

## Run the Complete Workflow

In [ ]:
await run_magentic_workflow()

## Expected Output Pattern

```
Building Magentic Workflow...

Task: I am preparing a report on the energy efficiency...

Starting workflow execution...

[ORCH:planning]
Task decomposition:
1. Research energy consumption data for ResNet-50, BERT-base, GPT-2
2. Calculate CO2 emissions for 24h training on Azure NC6s_v3
3. Create comparison tables
4. Provide recommendations
--------------------------

[STREAM:ResearcherAgent]: Finding energy consumption data for ML models...

[AGENT:ResearcherAgent] assistant
ResNet-50 training: ~50 kWh, BERT-base: ~1500 kWh, GPT-2: ~280 kWh...
--------------------------

[STREAM:CoderAgent]: Calculating CO2 emissions using Azure VM specs...

[AGENT:CoderAgent] assistant
| Model | Training Energy | CO2 Emissions |
|-------|----------------|---------------|
| ResNet-50 | 50 kWh | 25 kg CO2 |
...
--------------------------

==================================================
FINAL RESULT:
==================================================
Energy Efficiency Report:

Recommendations:
- Image Classification: ResNet-50 (most efficient)
- Text Classification: BERT-base (best accuracy/energy trade-off)
- Text Generation: GPT-2 (moderate energy consumption)
==================================================
```

## Key Takeaways

### 1. Magentic vs. Other Orchestration Patterns

| Pattern | When to Use | Key Feature |
|---------|------------|-------------|
| **Concurrent** | Independent parallel tasks | Fan-out/fan-in |
| **Sequential** | Linear, dependent steps | Shared context |
| **Magentic** | Complex, adaptive decomposition | Intelligent orchestration |

### 2. MagenticBuilder Configuration

```python
workflow = (
    MagenticBuilder()
    .participants(agent1=agent1, agent2=agent2)  # Named agents
    .on_event(callback, mode=MagenticCallbackMode.STREAMING)  # Events
    .with_standard_manager(  # Orchestrator config
        chat_client=client,
        max_round_count=10,
        max_stall_count=3,
        max_reset_count=2
    )
    .build()
)
```

### 3. Event Types and Handling

#### MagenticOrchestratorMessageEvent
- Planning decisions
- Task decomposition
- Progress updates

#### MagenticAgentDeltaEvent
- Streaming tokens from agents
- Real-time response generation
- Incremental display

#### MagenticAgentMessageEvent
- Complete agent responses
- Final message content
- Role and authorship

#### MagenticFinalResultEvent
- Synthesized workflow result
- Consolidated answer
- Completion signal

### 4. Callback Modes

- **STREAMING**: Receive incremental updates (MagenticAgentDeltaEvent)
- **NON_STREAMING**: Only complete messages (no deltas)

### 5. Standard Manager Parameters

- **max_round_count**: Prevents infinite loops
- **max_stall_count**: Retries when no progress
- **max_reset_count**: Full plan regeneration attempts

### 6. Agent Specialization

Magentic works best with clearly specialized agents:
- **Clear descriptions**: Help orchestrator choose correctly
- **Focused instructions**: Define agent capabilities
- **Appropriate tools**: Match agent purpose

### 7. Use Cases for Magentic

- **Research + Analysis**: Combine information gathering with computation
- **Multi-step Reports**: Complex tasks requiring planning
- **Adaptive Workflows**: Task decomposition not known upfront
- **Mixed Capabilities**: Agents with different tool sets

### 8. Production Considerations

- Monitor orchestration costs (additional LLM calls for planning)
- Set appropriate max_round_count to control costs
- Log orchestrator decisions for debugging
- Consider caching for repeated task patterns
- Add exception handling in callbacks
- Test with various task complexities

### 9. Advanced Patterns

- **With Checkpointing**: See magentic_checkpoint.ipynb
- **With Plan Review**: See magentic_human_plan_update.ipynb
- **Custom Managers**: Implement your own orchestration logic
- **Dynamic Agent Registration**: Add agents during execution